## Loading the dataset 

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [14]:
import tensorflow as tf

input_x = 784 
hidden_1 = 512
hidden_2 = 512
n_classes = 10
dropout = 0.8
test_valid_size = 256
batch_size = 128
epochs = 10
learning_rate = 0.001

weights = {
    'wh1': tf.Variable(tf.random_normal([input_x, hidden_1])),
    'wh2': tf.Variable(tf.random_normal([hidden_1, hidden_2])),
    'out': tf.Variable(tf.random_normal([hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([hidden_1])),
    'b2': tf.Variable(tf.random_normal([hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [20]:
def MLP(x, weights, biases, dropout):
    layer_1 = tf.add(tf.matmul(x, weights['wh1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, dropout)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['wh2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2, dropout)
    
    out = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    #out = tf.nn.softmax(out)
    
    return out

In [21]:
x = tf.placeholder(tf.float32, [None, input_x])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

logits = MLP(x, weights, biases, keep_prob)

cost = tf.reduce_mean(\
                        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout
            })
            
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.0
            })
            
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.0
            })
        
        print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))
    
    #Testing the model
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.0
    })
    
    print('Test accuracy: {}'.format(test_acc))

Epoch  1, Batch 429 -Loss:   152.5699 Validation Accuracy: 0.898438
Epoch  2, Batch 429 -Loss:    65.5990 Validation Accuracy: 0.917969
Epoch  3, Batch 429 -Loss:    73.7273 Validation Accuracy: 0.925781
Epoch  4, Batch 429 -Loss:    33.2053 Validation Accuracy: 0.933594
Epoch  5, Batch 429 -Loss:    21.1874 Validation Accuracy: 0.941406
Epoch  6, Batch 429 -Loss:    39.8663 Validation Accuracy: 0.941406
Epoch  7, Batch 429 -Loss:    38.1468 Validation Accuracy: 0.941406
Epoch  8, Batch 429 -Loss:    39.3060 Validation Accuracy: 0.941406
Epoch  9, Batch 429 -Loss:    30.4800 Validation Accuracy: 0.941406
Epoch 10, Batch 429 -Loss:    18.1359 Validation Accuracy: 0.937500
Test accuracy: 0.9453125
